# Triage-at-home
A notebook demonstrating machine learning and NLP capabilities such as NLTK in the domain of primary care.

In [31]:
# coding: utf-8
import codecs
#import csv

icpc_dict = {}
#icpc_dict_reverse = {}

f = open('resources/icpc-2e-v5-se.csv', 'r', encoding='utf-8')
for line in f:
    icpc_dict[line.partition(';')[0]] = line.partition(';')[2].replace('\n', '').lower()
    #icpc_dict_reverse[line.partition(';')[2].replace('\n', '').lower()] = line.partition(';')[0] # used when looking for the string and not the key
    
print(icpc_dict)
print('tjo')

{'\ufeff-30': 'medicinsk undersökning / grundlig', '-31': 'medicinsk undersökning / partiell', '-32': 'allergitest', '-33': 'mikrobiologiskt /annat immunologiskt test', '-34': 'blodprov', '-35': 'urinprov', '-36': 'avföringsprov', '-37': 'histologiskt / cytologiskt prov', '-38': 'annat laboratorieprov uns', '-39': 'kliniskt funktionstest', '-40': 'diagnostisk endoskopi', '-41': 'radiologisk diagnostik eller annan bilddiagnostik', '-42': 'elektrofysiologisk undersökning', '-43': 'annan diagnostisk åtgärd', '-44': 'preventiv immunisering / medicinering', '-45': 'observation / hälsofostran / rådgivning / dietrådgivning', '-46': 'konsultation med person yrkesverksam inom primärvården', '-47': 'specialistkonsultation', '-48': 'utredning av patientens kontaktorsak / -behov', '-49': 'annan förebyggande åtgärd', '-50': 'medicinering /ordination / förnyelse / injektion', '-51': 'incision / dränage/ sköljning / aspiration', '-52': 'excision / avlägsnande av vävnad/biopsi / destruktion / revision

## Processing the strings into words
A word-dictionary with the ICPC-code as (a potentially duplicate) key and interesting words (not stopwords) as value. Only one word per dictionary entry.

In [32]:
#sorted(icpc_dict.keys())
#print('-32' in icpc_dict.keys())
# print('kliniskt funktionstest' in icpc_dict_reverse.keys())
import nltk
from nltk.stem.snowball import SwedishStemmer
# nltk.download('stopwords')

swedish_stemmer = SwedishStemmer()	#(ignore_stopwords=False)
# print(swedish_stemmer.stem('snuvig'))

def remove_stopwords(s):
    stopwords = set(w.rstrip() for w in open('resources/stopwords_se.txt'))

    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    # tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

s = "Haft hosta under den senaste tiden. Är rökare och har känt sig snuvig i faktiskt flera månader och har även misstanke om någon form av allergi"
s = remove_stopwords(s)

tags = nltk.pos_tag(s)
# print(tags)
print(nltk.ne_chunk(tags))

# slå upp ovanstående ord i sina grundformer mot en lista av ord från ICPC
#for k, v in icpc_dict.items():
#    print(k, v)

(S
  haft/NN
  hosta/NN
  senaste/NN
  tiden/NN
  rökare/NN
  känt/FW
  snuvig/NN
  faktiskt/NN
  flera/NN
  månader/NN
  även/NNP
  misstanke/VBZ
  form/NN
  allergi/NN)
